In [1]:
import pandas as pd
# import os
from datetime import datetime, timedelta

import os, sys
module_path = os.path.abspath(os.path.join('./download-tick-from-dukascopy'))
if module_path not in sys.path:
    sys.path.append(module_path)
import download_tick_from_dukascopy

path = 'data'
inst_txt_path = 'insts.txt'

In [3]:
# inst = 'BTCUSD'
inst = 'USA500IDXUSD'
# inst = 'USATECHIDXUSD'
if inst not in os.listdir(path):
    os.mkdir('{}/{}'.format(path, inst))
    
#
#   To do:
#       - Check if already in data
#       - Download days you don't have
#
    
# gotta loop through this, and handle no columns (1kb .csv)

# check handling quitting

base = datetime.today()
date_list = [datetime.strftime(base - timedelta(days=x), '%Y-%m-%d') for x in range(0, 7)]
for date in date_list:
    download_tick_from_dukascopy.main(inst, date, date, './{}/{}'.format(path, inst))

downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/08/00h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/08/01h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/08/02h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/08/03h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/08/04h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/08/05h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/08/06h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/08/07h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/08/08h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/08/09h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/08/10h_ticks.bi5

downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/05/20h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/05/21h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/05/22h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/USA500IDXUSD/2021/03/05/23h_ticks.bi5


In [ ]:
# pd.read_csv(path + '/BTCUSD_2021-01-05_2021-01-06.csv')

In [4]:
def check_date():
    
    # check current date
    
    
    # open list of inst's to update
        # make insts.txt
    
    # for inst in insts:
        # open saved data (from list of inst's to update), find last date
    
    # if not updated, return inst and start date
    
    return datetime.now()

In [5]:
check_date()

datetime.datetime(2021, 4, 11, 22, 38, 6, 357386)

In [6]:
with open(inst_txt_path, 'r') as f:
    insts = f.read().split('\n')
insts

['GBPUSD', 'BTCUSD', 'USA500IDXUSD', 'USATECHIDXUSD']

In [17]:
[file for file in os.listdir('{}/{}'.format(path, inst)) if os.path.getsize('{}/{}/{}'.format(path, inst, file)) > 2]

['data/USA500IDXUSD/USA500IDXUSD_2021-04-05_2021-04-05.csv',
 'data/USA500IDXUSD/USA500IDXUSD_2021-04-06_2021-04-06.csv',
 'data/USA500IDXUSD/USA500IDXUSD_2021-04-07_2021-04-07.csv',
 'data/USA500IDXUSD/USA500IDXUSD_2021-04-08_2021-04-08.csv',
 'data/USA500IDXUSD/USA500IDXUSD_2021-04-09_2021-04-09.csv']

In [21]:
def get_all(inst):
    # concat all .csvs within dir
    data = pd.DataFrame()
    
    files = [
        file for file 
        in os.listdir('{}/{}'.format(path, inst)) 
        if os.path.getsize('{}/{}/{}'.format(path, inst, file)) > 2
    ]
    
    for csv in files:
        data = pd.concat([
            data,
            pd.read_csv(
                '{}/{}/{}'.format(path, inst, csv),
                header=None
            )
        ])
    data.columns = ['dt', 'bid', 'ask', 'bidvol', 'askvol']
    return data

In [22]:
def clean(inst):
    data = get_all(inst)
    # dd = len(data)
    data = data.drop_duplicates()
    # print('{} duplicate rows dropped'.format(dd-len(data)))
    # data = data.sort_values('dt')
    # print('max: {}'.format(data['dt'].max()))
    data['mid'] = (data['bid'] + data['ask']) / 2
    data.index = data['dt'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))
    data = data[['mid']].resample('1Min').ohlc()
    return data

In [23]:
# inst = 'USATECHIDXUSD'
# inst = 'BTCUSD'
inst = 'USA500IDXUSD'

data = clean(inst)
data

mid                                 
                          open       high        low      close
dt                                                             
2021-04-05 00:00:00  4042.5660  4042.7075  4040.9685  4042.1985
2021-04-05 00:01:00  4042.3465  4042.5045  4040.9985  4040.9985
2021-04-05 00:02:00  4041.0950  4041.2075  4040.6970  4040.9925
2021-04-05 00:03:00  4040.8505  4041.0075  4040.5105  4040.9955
2021-04-05 00:04:00  4040.8490  4041.9955  4040.7045  4041.9955
...                        ...        ...        ...        ...
2021-04-09 20:10:00  4120.6730  4120.9000  4120.4000  4120.7075
2021-04-09 20:11:00  4120.7660  4121.1940  4119.7030  4120.2015
2021-04-09 20:12:00  4120.2990  4120.7045  4120.2990  4120.6715
2021-04-09 20:13:00  4120.5460  4120.9255  4119.3955  4120.7030
2021-04-09 20:14:00  4120.5415  4121.7330  4120.3775  4121.2505

[6975 rows x 4 columns]

In [ ]:
# take last datetime, find difference from current date
(datetime.now() - datetime.strptime(data['dt'].max(), '%Y-%m-%d %H:%M:%S.%f')).days

In [ ]:
# just get last week because it's less annoying
datetime.strftime((datetime.now() - timedelta(7)), '%Y-%m-%d')

In [ ]:
# class insts list

# class inst
    # start date
    # end date
    # write to csv
    # do things

In [ ]:
for inst in insts:
    print(inst)
    if inst not in os.listdir(path):
        os.mkdir('{}/{}'.format(path, inst))
    print(os.listdir('{}/{}'.format(path, inst)))
    
    # consider making some meta-file that keeps last updated date instead of concatting everytime
    
    